In this notebook I continue and improve on the work by Mr. Kireev from his great DL [here](https://www.kaggle.com/alexanderkireev/deep-learning-support-9663). All credit goes to him, I have simply tuned hyperparameters and messed around with the layers of the network (+ some validation).

First we will read the data, starting after row 131886954.

In [1]:
import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['OMP_NUM_THREADS'] = '4'

dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

print('loading')

train_columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_columns = ['click_id', 'ip', 'app', 'device', 'os', 'channel', 'click_time']

path = ''
path_old = 'old/'
path_two = "mnt/ssd/kaggle-talkingdata2/competition_files/"
path_train = path_two + 'train.csv'
path_test = path + 'test.csv'
path_old_test = path_old + 'old_test.csv'


train_df = pd.read_csv(path+"train.csv", dtype=dtypes, usecols=train_columns)
test_df = pd.read_csv(path_old_test, dtype=dtypes, usecols=test_columns)
print('finished loading')

/home/dcek/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


loading
finished loading


In [2]:
train_df.head()

,ip,app,device,os,channel,click_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,0
1,17357,3,1,19,379,2017-11-06 14:33:34,0
2,35810,3,1,13,379,2017-11-06 14:34:12,0
3,45745,14,1,13,478,2017-11-06 14:34:52,0
4,161007,3,1,13,379,2017-11-06 14:35:08,0


The preprocessing is the same as the [original kernel](https://www.kaggle.com/alexanderkireev/deep-learning-support-9663).

In [3]:
def prep_data(d):
    print('hour, day, wday....')
    d['hour'] = pd.to_datetime(d.click_time).dt.hour.astype('uint8')
    d['day'] = pd.to_datetime(d.click_time).dt.day.astype('uint8')
    d['wday']  = pd.to_datetime(d.click_time).dt.dayofweek.astype('uint8')
    print('grouping by ip-day-hour combination....')
    gp = d[['ip', 'day', 'hour', 'channel']].groupby(by=['ip', 'day', 'hour'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'qty'})
    d = d.merge(gp, on=['ip', 'day', 'hour'], how='left')
    del gp; gc.collect()
    print('group by ip-app combination....')
    gp = d[['ip', 'app', 'channel']].groupby(by=['ip', 'app'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_count'})
    d = d.merge(gp, on=['ip', 'app'], how='left')
    del gp; gc.collect()
    print('group by ip-app-os combination....')
    gp = d[['ip', 'app', 'os', 'channel']].groupby(by=['ip', 'app', 'os'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_os_count'})
    d = d.merge(gp, on=['ip', 'app', 'os'], how='left')
    del gp; gc.collect()
    print("vars and d type....")
    d['qty'] = d['qty'].astype('uint16')
    d['ip_app_count'] = d['ip_app_count'].astype('uint16')
    d['ip_app_os_count'] = d['ip_app_os_count'].astype('uint16')
    print("label encoding....")
    from sklearn.preprocessing import LabelEncoder
    d[['app', 'device', 'os', 'channel', 'hour', 'day', 'wday']].apply(LabelEncoder().fit_transform)
    print('dropping')
    #d.drop(['click_time', 'ip'], 1, inplace=True)
    
    return d

In [4]:
train_df = prep_data(train_df)
test_df = prep_data(test_df)
print("finished")

hour, day, wday....
grouping by ip-day-hour combination....
group by ip-app combination....
group by ip-app-os combination....
vars and d type....
label encoding....
dropping
hour, day, wday....
grouping by ip-day-hour combination....
group by ip-app combination....
group by ip-app-os combination....
vars and d type....
label encoding....
dropping
finished


We will keep 10% of the training dataset for validation.

In [5]:
import gc
from datetime import datetime
VALIDATE = True
RANDOM_SEED = 1
import random
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

#predictors =  ['app','channel','click_time', 'device','ip','is_attributed', 'os', 'hour', 'day','ip_app_channel_var_day', 'n_channels', 'ip_app_count', 'ip_app_os_count', 'qty_var','ip_app_os_var','ip_app_channel_mean_hour','ip_nextClick','ip_app_nextClick','ip_channel_nextClick','ip_os_nextClick','clicks_by_ip','nip_hh_os','nip_hh_app','nip_hh_dev', 'prev_app_clicks','future_app_clicks','prev_identical_clicks','future_identical_clicks','nip_day_test_hh','ip_prevClick','ip_app_prevClick']
if VALIDATE:
    train_df['click_time']= pd.to_datetime(train_df['click_time'])
    start_date = datetime(2017,11,9,4,0,0)
    end_date = datetime(2017,11,9,15,0,0)
    maskval = (train_df['click_time'] >= start_date) & (train_df['click_time'] <= end_date)
    masktrain = (train_df['click_time'] < start_date)
    
    r = 0.1 # the fraction of the train data to be used for validation
    #val = train[(len_train-round(r*len_train)):len_train]
    X_test = train_df.loc[maskval]
    print('The size of the validation set is ', len(X_test))
    
    #train = train[:(len_train-round(r*len_train))]
    X_train = train_df.loc[masktrain]
    print('The size of the train set is ', len(X_train))
    
    y_train = X_train['is_attributed']
    X_train = X_train.drop(['is_attributed'], axis=1)
    y_test = X_test['is_attributed']
    X_test = X_test.drop(['is_attributed'], axis=1)

The size of the validation set is  37170060
The size of the train set is  144708152


In [6]:
from keras.layers import Input, Embedding, Dense, Flatten, Dropout, concatenate
from keras.layers import BatchNormalization, SpatialDropout1D, Conv1D
from keras.callbacks import Callback
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [7]:
X_train.head()

,ip,app,device,os,channel,click_time,hour,day,wday,qty,ip_app_count,ip_app_os_count
0,83230,3,1,13,379,2017-11-06 14:32:21,14,6,0,1,5427,1326
1,17357,3,1,19,379,2017-11-06 14:33:34,14,6,0,1,5025,1380
2,35810,3,1,13,379,2017-11-06 14:34:12,14,6,0,1,2012,442
3,45745,14,1,13,478,2017-11-06 14:34:52,14,6,0,1,9474,1954
4,161007,3,1,13,379,2017-11-06 14:35:08,14,6,0,1,232,80


In [8]:
def get_keras_data(dataset):
    X = {
        'app': np.array(dataset.app),
        'ch': np.array(dataset.channel),
        'dev': np.array(dataset.device),
        'os': np.array(dataset.os),
        'h': np.array(dataset.hour),
        'd': np.array(dataset.day),
        'wd': np.array(dataset.wday),
        'qty': np.array(dataset.qty),
        'c1': np.array(dataset.ip_app_count),
        'c2': np.array(dataset.ip_app_os_count)
    }
    return X

train = get_keras_data(X_train)
test = get_keras_data(X_test)

In [9]:
max_app = np.max([train_df['app'].max(), test_df['app'].max()])+1
max_ch = np.max([train_df['channel'].max(), test_df['channel'].max()])+1
max_dev = np.max([train_df['device'].max(), test_df['device'].max()])+1
max_os = np.max([train_df['os'].max(), test_df['os'].max()])+1
max_h = np.max([train_df['hour'].max(), test_df['hour'].max()])+1
max_d = np.max([train_df['day'].max(), test_df['day'].max()])+1
max_wd = np.max([train_df['wday'].max(), test_df['wday'].max()])+1
max_qty = np.max([train_df['qty'].max(), test_df['qty'].max()])+1
max_c1 = np.max([train_df['ip_app_count'].max(), test_df['ip_app_count'].max()])+1
max_c2 = np.max([train_df['ip_app_os_count'].max(), test_df['ip_app_os_count'].max()])+1

In [10]:
emb_n = 50

in_app = Input(shape=[1], name = 'app')
emb_app = Embedding(max_app, emb_n)(in_app)
in_ch = Input(shape=[1], name = 'ch')
emb_ch = Embedding(max_ch, emb_n)(in_ch)
in_dev = Input(shape=[1], name = 'dev')
emb_dev = Embedding(max_dev, emb_n)(in_dev)
in_os = Input(shape=[1], name = 'os')
emb_os = Embedding(max_os, emb_n)(in_os)
in_h = Input(shape=[1], name = 'h')
emb_h = Embedding(max_h, emb_n)(in_h) 
in_d = Input(shape=[1], name = 'd')
emb_d = Embedding(max_d, emb_n)(in_d) 
in_wd = Input(shape=[1], name = 'wd')
emb_wd = Embedding(max_wd, emb_n)(in_wd) 
in_qty = Input(shape=[1], name = 'qty')
emb_qty = Embedding(max_qty, emb_n)(in_qty) 
in_c1 = Input(shape=[1], name = 'c1')
emb_c1 = Embedding(max_c1, emb_n)(in_c1) 
in_c2 = Input(shape=[1], name = 'c2')
emb_c2 = Embedding(max_c2, emb_n)(in_c2) 
fe = concatenate([(emb_app), (emb_ch), (emb_dev), (emb_os), (emb_h), 
                 (emb_d), (emb_wd), (emb_qty), (emb_c1), (emb_c2)])

Below we will build the main core of the network. If you want to mess around, I suggest you start by adjusting the `Dropout` parameters. I have noticed that up to 0.5 it works really well.

In [11]:
s_dout = SpatialDropout1D(0.25)(fe)

fl1 = Flatten()(s_dout)
conv1 = Conv1D(70, kernel_size=2, strides=1, padding='same')(s_dout)
fl2 = Flatten()(conv1)
conv2 = Conv1D(70, kernel_size=4, strides=1, padding='same')(s_dout)
fl3 = Flatten()(conv2)

concat = concatenate([(fl1), (fl2), (fl3)])

x = Dropout(0.2)(Dense(500, activation='relu')(concat))
x = Dropout(0.3)(Dense(750, activation='relu')(x))
x = Dropout(0.4)(Dense(1000, activation='relu')(x))
outp = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[in_app,in_ch,in_dev,in_os,in_h,in_d,in_wd,in_qty,in_c1,in_c2], outputs=outp)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


Time to train our network!

Possible improvements include increasing `batch_size` and the adjusting the `lr_decay` values.

In [12]:
batch_size = 25000
epochs = 1
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(len(train_df) / batch_size) * epochs
lr_init, lr_fin = 0.0025, 0.0001
lr_decay = exp_decay(lr_init, lr_fin, steps)
optimizer_adam = Adam(lr=0.0025, decay=lr_decay)
model.compile(loss='binary_crossentropy', optimizer=optimizer_adam, metrics=['accuracy'])

In [13]:
from keras.callbacks import EarlyStopping

epochs =10
earlystop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, \
                          verbose=1, mode='auto')
callbacks_list = [earlystop]

In [14]:
class_weight = {0:.01,1:.99} # magic
model.fit(train, y_train, batch_size=batch_size, epochs=epochs, class_weight=class_weight, validation_data=(test, y_test),callbacks=callbacks_list, verbose=1)

Train on 144708152 samples, validate on 37170060 samples
Epoch 1/10
144708152/144708152 [==============================] - 741s 5us/step - loss: 0.0016 - acc: 0.9875 - val_loss: 0.0771 - val_acc: 0.9843
Epoch 2/10
144708152/144708152 [==============================] - 730s 5us/step - loss: 0.0015 - acc: 0.9880 - val_loss: 0.0748 - val_acc: 0.9845
Epoch 3/10
144708152/144708152 [==============================] - 733s 5us/step - loss: 0.0015 - acc: 0.9881 - val_loss: 0.0652 - val_acc: 0.9840
Epoch 4/10
144708152/144708152 [==============================] - 734s 5us/step - loss: 0.0014 - acc: 0.9881 - val_loss: 0.0671 - val_acc: 0.9832
Epoch 5/10
144708152/144708152 [==============================] - 730s 5us/step - loss: 0.0014 - acc: 0.9881 - val_loss: 0.0731 - val_acc: 0.9825
Epoch 6/10
  1250000/144708152 [..............................] - ETA: 25:58 - loss: 0.0014 - acc: 0.9880

KeyboardInterrupt: 

In [ ]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [15]:
test_df = get_keras_data(test_df)

In [16]:
pred = model.predict(test_df, batch_size=batch_size, verbose=1)

57537505/57537505 [==============================] - 111s 2us/step


In [17]:

path = ''
path_old = 'old/'
path_two = "mnt/ssd/kaggle-talkingdata2/competition_files/"
path_train = path + 'train.csv'
path_test = path + 'test.csv'
path_old_test = path_old + 'old_test.csv'

hello = pd.read_csv(path_test, dtype='int', usecols=['click_id'])


In [18]:
mapping = pd.read_csv('mapping.csv', dtype={'click_id': 'int32','old_click_id': 'int32'}, engine='c',
                na_filter=False,memory_map=True)

In [19]:
del test
gc.collect()

15868

In [20]:
submit = pd.read_csv(path_old_test, dtype='int', usecols=['click_id'])

print("Predicting the submission data...")

submit['is_attributed'] = pred

Predicting the submission data...


In [21]:
final_pred = pd.merge(hello, mapping, on='click_id')
final_pred = final_pred.rename(columns={'click_id':'temp', 'old_click_id': 'click_id'})
final_pred_final = pd.merge(final_pred, submit, on='click_id')
final_pred_final.drop('click_id', axis=1, inplace=True )
final_pred_final = final_pred_final.rename(columns={'temp':'click_id'})
final_pred_final.to_csv('predict_feature.csv', index=False)

## Validation

Here we will perform some basic validation on 10% of the collected training data set. We will calculate the ROC-AUC score, from this script [here](https://www.kaggle.com/antmarakis/calculating-and-plotting-auc-score).

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support)

fpr, tpr, thresholds = roc_curve(y_test, predictions)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.001, 1])
plt.ylim([0, 1.001])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show();

## Submission

Finally, creating the submission file:

In [ ]:
ids = test_df['click_id']
test_df.drop('click_id', axis=1, inplace=True)
t = get_keras_data(test_df)
sub_preds = model.predict(t, batch_size=batch_size, verbose=1)

In [ ]:
sub = pd.DataFrame()
sub['click_id'] = ids
sub['is_attributed'] = sub_preds
sub.to_csv('dl_sub.csv', index=False)